In [1]:
using Compose,Gadfly

set_default_plot_size(12cm, 12cm)


In [2]:
function GetReach(v,Adj,Labels)
    
    n = size(Adj,1)
    reach = spzeros(n,1)
    
    X = sparsevec([v],[1],n);
    marker = spzeros(n,1)

    #iter is just a statistic counter
    iter = 0
    while( nnz(X)>0)
        Xn = *(Adj,X);  
        rows = find(X)
        marker[rows] = 1
        
        for i in find(Xn)
            reach[i] =  Xn[i] * (1-max(marker[i],Labels[i])>0)
        end
        
        X=spzeros(n,1)
        for i in find(Xn)
            X[i]= Xn[i] * (1- max(marker[i],reach[i])>0)
        end
                
        iter+=1
    end
          
    return (reach,iter)
end

GetReach (generic function with 1 method)

In [32]:
function GetReaches(vs,Adj,Labels)
    nv = length(vs)
    n = size(Adj,1)
    reaches = spzeros(n,nv)
    
    XS = sparse(vs,[1:nv],vec(ones(1,nv)),n,n)
    
    markers = spzeros(n,nv)


    #iter is just a statistic counter
    iter = 0
    while( nnz(XS)>0)
        (rx,cx)=findn(XS)
        for ir =1:length(rx)
            i = rx[ir]
            j = cx[ir]
            markers[i,j] = 1
        end
        
        Xns = *(Adj,XS);  
        
        (r,c)=findn(Xns)

        for ir =1:length(r)
            i = r[ir]
            j = c[ir]
          #reaches[i,j] +=  (Xns[i,j] * (1-max(markers[i,j],Labels[i])>0))
          reaches[i,j] +=  (Xns[i,j] * (1-max(markers[i,j],Labels[i])>0))
        end
        
        
        XS=spzeros(n,nv)
        for ir =1:length(r)
            i = r[ir]
            j = c[ir]
                XS[i,j]= (Xns[i,j] * (1- max(markers[i,j],reaches[i,j])>0))
        end    

        iter+=1
    end
        

    return (reaches,iter)
end


GetReaches (generic function with 1 method)

In [54]:
function GetReaches1(vs,Adj,Labels)
    nv = length(vs)
    n = size(Adj,1)
    reaches = spzeros(n,nv)
    
    k = 1
    iter = -1
    for s in vs
        (rs,it) = GetReach(s,Adj,Labels)
        reaches[:,k] = rs
        k = k +1
        iter = max(iter, it)
    end
    return (reaches,iter)
end


GetReaches1 (generic function with 1 method)

In [72]:
function MD(B)

    n = size(B,1)
    Labels = vec(zeros(n,1))
    #statistics
    SPMV = vec(zeros(n,1))
    SPMM = vec(zeros(n,1))
    SPMM_SIZE = vec(zeros(n,1))

    #initialize degree list
    Degrees = sum(B,2)-ones(n,1);

    for s=1:n
        #pick node with minimum degree
        print(Degrees')
        print("\n")
        mind = n+1
        v = -1
        for i=1:length(Degrees)
            if Labels[i]==0
                if Degrees[i]<mind
                    v = i
                    mind = Degrees[i]
                end
            end
        end

        #get the reachable set of the minimum degree node v
        iter = 0;
        (reach,iter) = GetReach(v,B,Labels)
        Labels[v]=s
        assert(Degrees[v]==nnz(reach))
        #statistics
        SPMV[s] = iter
        
        #us is the list of nodes reachable by v
        us = find(reach) 
        iter = 0;
        (reaches_u,iter) = GetReaches1(us,B,Labels)
        for i=1:length(us)
            u = us[i]
            Degrees[u] = nnz(reaches_u[:,i])
        end
        #statistics
        SPMM[s]=iter
        SPMM_SIZE[s]=length(us)

    end
  
    return (Labels,SPMV,SPMM,SPMM_SIZE)
end

MD (generic function with 1 method)

In [73]:
using MatrixDepot


A = spones(matrixdepot("HB/bcsstk01", :r));
#A = spones(matrixdepot("nasa2146", :r))
n = max(size(A,1),size(A,2));
B = spones(A'+A);




@time (Labels,SPMV,SPMM,SPMM_SIZE) = MD(B);

    print("SPMV / SPMM / SPMM_SIZE count is\n")
    for i=1:n
        print(int(i),": ",int(SPMV[i])," ",int(SPMM[i])," ",int(SPMM_SIZE[i]),"\n")
    end
    print("\n")

print("Labels computed","\n")



[7.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 10.0 11.0 11.0 10.0 11.0 11.0 7.0 7.0 7.0 7.0 7.0 7.0 6.0 4.0 4.0 6.0 4.0 4.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 9.0 8.0 8.0 9.0 8.0 8.0]
[7.0 9.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 10.0 11.0 11.0 10.0 11.0 11.0 7.0 7.0 7.0 7.0 7.0 7.0 6.0 4.0 4.0 6.0 4.0 4.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 9.0 8.0 8.0 9.0 8.0 8.0]
[7.0 9.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 10.0 11.0 11.0 10.0 11.0 11.0 7.0 7.0 7.0 7.0 7.0 7.0 6.0 4.0 4.0 6.0 4.0 4.0 7.0 7.0 9.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 9.0 8.0 8.0 9.0 8.0 8.0]
[7.0 9.0 7.0 7.0 9.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 10.0 11.0 11.0 10.0 11.0 11.0 7.0 7.0 7.0 7.0 7.0 7.0 6.0 4.0 4.0 6.0 4.0 4.0 7.0 7.0 9.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 9.0 8.0 8.0 9.0 8.0 8.0]
[7.0 9.0 7.0 7.0 9.0 7.0 7.0 7.0 7.0 7.0 7.0 7.0 10.0 11.0 11.0 10.0 11.0 11.0 7.0 7.0 7.0 7.0 7.0 7.0 6.0 4.0 4.0 6.0 4.0 4.0 7.0 7.0 9.0 7.0 7.0 9.0 7.0 7.0 7.0 7.0 7.0 7.0 9.0 8.0 8.0 9.0 8.0 8.0]


In [74]:
#print(Labels)
n = length(Labels)
perm = zeros(n,1)
for i=1:n
    lab = Labels[i]
    perm[lab]=i
end
print("Perm is\n")
for i in perm
    print(int(i)," ")
end
print("\n")



Perm is
26 27 29 30 25 28 7 8 9 19 20 21 37 38 39 47 4 43 48 6 13 10 34 45 46 1 24 31 2 3 5 11 12 14 15 16 17 18 22 23 32 33 35 36 40 41 42 44 


In [70]:
csvfile = open("bcsstk01_stats.csv","w")
@printf(csvfile,"step,SPMV,SPMM,SPMM_SIZE\n")
    for i=1:n
    @printf(csvfile,"%d, %d, %d, %d\n",int(i),int(SPMV[i]),int(SPMM[i]),int(SPMM_SIZE[i]))
    end
    close(csvfile)


In [2]:
a = spzeros(6,4)
nnz(a)

0